<a href="https://colab.research.google.com/github/pedrohortencio/text-generator/blob/main/Song%20Generator/Gerando_M%C3%BAsicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup
#@markdown _Execute essa célula primeiro. Duplo clique exibe o código._
 
print("Iniciando")

TOTAL_WORDS_BASE = 3071
MAX_SEQ_LEN_BASE = 83
TOTAL_WORDS_ATT = 6760
MAX_SEQ_LEN_ATT = 152

import numpy as np
from IPython.utils import io

print("Configurando o Keras")
### Reinstalando o Keras
with io.capture_output() as captured:
    !pip uninstall keras -y
    !pip install -U keras

### Baixando o arquivo generic_utils.py diretamente do repositório do TensorFlow
import os
from shutil import copyfile

path_utils = '/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py'
url_utils = 'https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/python/keras/utils/generic_utils.py'

with io.capture_output() as captured:
    os.remove(path_utils)
    !wget  $url_utils
    copyfile('/content/generic_utils.py', path_utils)

print("Adicionando Attention")
#### Instalando o módulo de self-attention
with io.capture_output() as captured:
    !pip install keras-self-attention
from tensorflow import keras
from keras_self_attention import SeqSelfAttention


#url = 'https://raw.githubusercontent.com/thushv89/attention_keras/322a16ee147122026b63305aaa5e899d9e5de883/src/layers/attention.py'
#!wget $url
#from attention import AttentionLayer

### Importando bibliotecas
import pickle
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, BatchNormalization, GlobalMaxPooling1D, GRU, Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


### URLs
tokenizer_base_url = "https://github.com/pedrohortencio/text-generator/raw/main/Song%20Generator/model-files/tokenizer14012021BASE.pickle"
tokenizer_url = "https://github.com/pedrohortencio/text-generator/raw/main/Song%20Generator/model-files/tokenizer13012021CONV1D.pickle"

model_base_url = "https://github.com/pedrohortencio/text-generator/raw/main/Song%20Generator/model-files/treino-14012021BASE.h5"
model_conv_url = "https://github.com/pedrohortencio/text-generator/raw/main/Song%20Generator/model-files/treino-13012021CONV1D.h5"
model_bilstmV2_url = "https://github.com/pedrohortencio/text-generator/raw/main/Song%20Generator/model-files/treino-13012021BiLSTM.h5"
model_bilstmV1_url = "https://github.com/pedrohortencio/text-generator/raw/main/Song%20Generator/model-files/treino-09062021BiLSTM.h5"
model_bigru_url = "https://github.com/pedrohortencio/text-generator/raw/main/Song%20Generator/model-files/treino-13012021BiGRU.h5"


import urllib
import urllib.request

print("Importando Tokenizers")

tokenizer_base_pth, _ = urllib.request.urlretrieve(tokenizer_base_url, filename="/content/tokenizer_base.pickle")
tokenizer_pth, _ = urllib.request.urlretrieve(tokenizer_url, filename="/content/tokenizer.pickle")

with open(tokenizer_base_pth, 'rb') as handle:
    tokenizer_base = pickle.load(handle)
with open(tokenizer_pth, 'rb') as handle:
    tokenizer = pickle.load(handle)

print("Criando Modelos")

print("1/5")
def biLSTMV1():
    EMBEDDING_SIZE = 300
    LSTM_UNITS = 128
    adam = Adam()

    model = Sequential()
    model.add(Embedding(TOTAL_WORDS_ATT, EMBEDDING_SIZE, input_length=MAX_SEQ_LEN_ATT-1, mask_zero=True))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(LSTM_UNITS, return_sequences=True)))
    model.add(SeqSelfAttention(attention_width=25,  attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='sigmoid'))
    model.add(GlobalMaxPooling1D())
    model.add(BatchNormalization())
    model.add(Dense(TOTAL_WORDS_ATT, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

model_bilstmV1_pth, _ = urllib.request.urlretrieve(model_bilstmV1_url, filename="/content/treino-09062021BiLSTM.h5")
biLSTMV1 = biLSTMV1()
biLSTMV1.load_weights(model_bilstmV1_pth)

print("2/5")
def biLSTMV2():
    EMBEDDING_SIZE = 300
    LSTM_UNITS = 150
    adam = Adam()

    model = Sequential()
    model.add(Embedding(TOTAL_WORDS_ATT, EMBEDDING_SIZE, input_length=MAX_SEQ_LEN_ATT-1, mask_zero=True))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(LSTM_UNITS, return_sequences=True)))
    model.add(SeqSelfAttention(attention_width=20,  attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='sigmoid'))
    model.add(GlobalMaxPooling1D())
    model.add(BatchNormalization())

    model.add(Dense(TOTAL_WORDS_ATT, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

model_bilstmV2_pth, _ = urllib.request.urlretrieve(model_bilstmV2_url, filename="/content/treino-13012021BiLSTM.h5")
biLSTMV2 = biLSTMV2()
biLSTMV2.load_weights(model_bilstmV2_pth)

print("3/5")
def biGRU():
    EMBEDDING_SIZE = 300
    LSTM_UNITS = 150
    adam = Adam()

    model = Sequential()
    model.add(Embedding(TOTAL_WORDS_ATT, EMBEDDING_SIZE, input_length=MAX_SEQ_LEN_ATT-1, mask_zero=True))
    model.add(BatchNormalization())

    model.add(Bidirectional(GRU(LSTM_UNITS, return_sequences=True)))
    model.add(SeqSelfAttention(attention_width=25,  attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='sigmoid'))
    model.add(GlobalMaxPooling1D())
    model.add(BatchNormalization())

    model.add(Dense(TOTAL_WORDS_ATT, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

model_bigru_pth, _ = urllib.request.urlretrieve(model_bigru_url, filename="/content/treino-13012021BiGRU.h5")
biGRU = biGRU()
biGRU.load_weights(model_bigru_pth)

print("4/5")
def m_conv1D():
    EMBEDDING_SIZE = 300
    model = Sequential()
    model.add(Embedding(TOTAL_WORDS_ATT, EMBEDDING_SIZE, input_length=MAX_SEQ_LEN_ATT-1, mask_zero=True))
    model.add(BatchNormalization())

    model.add(Conv1D(512, 10, padding='same', activation='relu'))
    model.add(SeqSelfAttention(attention_width=25, attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='sigmoid'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())

    model.add(Conv1D(256, 5, padding='same', activation='relu'))
    model.add(SeqSelfAttention(attention_width=15, attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='sigmoid'))
    model.add(GlobalMaxPooling1D())
    model.add(BatchNormalization())

    model.add(Dense(TOTAL_WORDS_ATT, activation='softmax'))

    adam = Adam()
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

model_conv_pth, _ = urllib.request.urlretrieve(model_conv_url, filename="/content/treino-13012021CONV1D.h5")
m_conv1D = m_conv1D()
m_conv1D.load_weights(model_conv_pth)

print("5/5")
def base_model():
    EMBEDDING_SIZE = 300
    LSTM_UNITS = 256
    LEARNING_RATE = 0.01

    model = Sequential()
    model.add(Embedding(TOTAL_WORDS_BASE, EMBEDDING_SIZE, input_length=MAX_SEQ_LEN_BASE-1))
    model.add(Bidirectional(LSTM(LSTM_UNITS)))
    model.add(Dense(TOTAL_WORDS_BASE, activation='softmax'))

    adam = Adam(learning_rate=LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

model_base_pth, _ = urllib.request.urlretrieve(model_base_url, filename="/content/treino-14012021BASE.h5")
base_model = base_model()
base_model.load_weights(model_base_pth)



def predict_base(seed_text, model, next_words=50):
    for _ in range(next_words):
        token_list = tokenizer_base.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=MAX_SEQ_LEN_BASE-1, padding='pre')
        #predicted = model.predict_classes(token_list, verbose=0)
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer_base.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    
    versos = seed_text.split(' <br> ')
    counter = -1
    value = 4
    print('\n Musica: ',versos[0],'\n')
    for v in versos[1:]:
        print(v.capitalize())
        counter = counter + 1
        if counter == value:
            print('\n')
            counter = 0

def predict_att(seed_text, model, next_words=50):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=MAX_SEQ_LEN_ATT-1, padding='pre')
        #predicted = model.predict_classes(token_list, verbose=0)
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    
    versos = seed_text.split(' <br> ')
    counter = -1
    value = 4
    print('\n Musica: ',versos[0],'\n')
    for v in versos[1:]:
        print(v.capitalize())
        counter = counter + 1
        if counter == value:
            print('\n')
            counter = 0

>Sugestões de frases (Por favor, digite sem aspas):
>
>* "Você é minha vida"
* "Amor da minha vida"
* "Já vi que você vai sair \<br> E a sequência eu já sei"
* "Ele não faz questão de pegar na sua mão \<br> Eu nunca vi ele te dar beijo na frente de ninguém \<br> Tem algo errado nessa relação e você sabe quem"
* "Barzinho outra vez"
* "Disk-recaída, bateu carência liga"
* "Agora não chega mais nenhuma notificação sua"
* "Você não é nada além de uma causa perdida"
* "Vou tentar te esquecer"
* "Seguir seus passos já não posso mais"
* "Nunca me enxerguei em outra pessoa"

In [ ]:
#@title Geração de Músicas
#@markdown _Execute quantas vezes quiser. Note que a execução pode demorar alguns minutos._
seed = input("Digite uma frase que servirá como o título da música gerada:\n")
qtd_palavras = int(input("Quantas palavras por música (default=50):\n"))

lista_modelos = [biLSTMV1, biLSTMV2, biGRU, m_conv1D, base_model]
desc_modelos = ["Bidirectional LSTM + Attention (V1)", "Bidirectional LSTM + Attention (V2)",
                "Bidirectional GRU + Attention", "Conv1D + Attention", "Bidirectional LSTM (Base)"]

for model, desc in zip(lista_modelos, desc_modelos):
    print("\n", "--"*25, )
    print("Modelo: ", desc, "\n")

    if model != base_model:
        predict_att(seed, model, qtd_palavras)
    else:
        predict_base(seed, model, qtd_palavras)

## Sobre os Modelos

O código fonte de criação dos modelos e seus respectivos pesos estão disponíveis no [GitHub](https://github.com/pedrohortencio/text-generator/tree/main/Song%20Generator).

São 5 modelos apresentados aqui, ao todo:
* Bidirectional LSTM (Base)
> Um modelo com um único layer LSTM bidirecional. Gera bons resultados, mas tem a tendência a repetir frases e palavras quando o número de palavras geradas é alto. Há, também, a tendência a simplesmente copiar trechos de músicas do conjunto de treinamento.

* Bidirectional LSTM + Attention (V1)
> É a implementação de self-attention no modelo anterior. A adição de self-attention visa resolver o problema de cópia e repetição de palavras.


* Bidirectional LSTM + Attention (V2)
> Pequenas modificações no modelo anterior. Conta com 150 unidades (em vez de 128) e foi treinado por uma maior quantidade de épocas.

* Bidirectional GRU + Attention
> Implementação que substitui o layer LSTM do modelo anterior por um layer GRU.

* Conv1D + Attention
> Assim como o modelo com GRU, essa implementação substitui a camada LSTM por duas camadas Conv1D, com a operação MaxPooling1D entre elas. Ambas as camadas possuem self-attention. Foi o modelo com o menor tempo de treinamento (de longe o menor, consumindo 40 minutos contra 1h-2h dos modelos anteriores).